In [1]:
#note: temp readings are 20s apart

import pandas as pd
import os
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
import numpy as np

os.chdir('data')

In [6]:
dfs = []# list of dataframes

for folder in os.listdir():
    if os.path.isdir(folder):
        os.chdir(folder)
        for file in os.listdir():
            try:
                df = pd.read_excel(file, sheet_name="DATA", usecols='C,D', skiprows = 1)
                df.loc[df.shape[0]] = [None, None]# to ensure that there are breaks between each recorded shower
                dfs.append(df)
            except:
                print(file)
        os.chdir('..')


data = pd.concat(dfs, ignore_index = True)# concatenating all data into one dataframe
data.columns = ['temp', 'label']

~$Copy of Dascomb 20461369 2019-12-02 18_07_58 -0500.xlsx


In [ ]:
start_pts = []#list where the first item is 1 for start and 0 for not start
end_pts = []#same list but for ends
for i, row in data.iterrows():
    if not pd.isnull(row['label']):
        if row['label'].startswith('Start'):
            start_pts.append([1] + data.iloc[i-4:i+5]['temp'].tolist())
        if row['label'].startswith('End'):
            end_pts.append([1] + data.iloc[i-4:i+5]['temp'].tolist())
            
initial_length = len(start_pts)
if initial_length != len(end_pts):
    print("inconsistent labels!")

random_start_indices = random.sample(range(len(data) - 9), k =  len(data) - 9)
random_end_indices = random.sample(range(len(data) - 9), k = len(data) - 9)

for i in random_start_indices:
    if not (any(pd.isnull(data.iloc[i:i+9]['temp'])) or
            (not pd.isnull(data.iloc[i+3]['label']) and data.iloc[i+3]['label'].startswith('Start'))):
        start_pts.append([0] + data.iloc[i:i+9]['temp'].tolist())
        if len(start_pts) == initial_length * 2:
            break

for i in random_end_indices:
    if not (any(pd.isnull(data.iloc[i:i+9]['temp'])) or
            (not pd.isnull(data.iloc[i+3]['label']) and data.iloc[i+3]['label'].startswith('End'))):
        end_pts.append([0] + data.iloc[i:i+9]['temp'].tolist())
        if len(end_pts) == initial_length * 2:
            break

In [ ]:
start_data = tf.convert_to_tensor(start_pts)#tensors holding the label, followed by a list of 10 temperatures
start_data = tf.random.shuffle(start_data)

end_data = tf.convert_to_tensor(end_pts)
end_data = tf.random.shuffle(end_data)

In [ ]:
data_length = start_data.shape[0]

start_train = start_data[:int(data_length * .7)]
start_val = start_data[int(data_length * .7):int(data_length * .85)]
start_test = start_data[int(data_length * .85):]

end_train = end_data[:int(data_length * .7)]
end_val = end_data[int(data_length * .7):int(data_length * .85)]
end_test = end_data[int(data_length * .85):]

In [ ]:
start_model = Sequential(
    [
        BatchNormalization(),
        Dense(16, activation = 'relu', input_shape=(10,)),
        Dense(16, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ]
)

start_model.compile(loss='mse', metrics=['mae'])
start_model.fit(start_train[:,1:], start_train[:,0], epochs=20)
start_val_predicted = start_model.predict(start_val[:,1:])

In [ ]:
np.mean(np.absolute(start_val_predicted[:,0] - start_val[:,0]))

In [ ]:
end_model = Sequential(
    [
        BatchNormalization(),
        Dense(16, activation = 'relu', input_shape=(10,)),
        Dense(16, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ]
)

end_model.compile(loss='mse', metrics=['mae'])
end_model.fit(end_train[:,1:], end_train[:,0], epochs=20)
end_val_predicted = end_model.predict(end_val[:,1:])

In [ ]:
np.mean(np.absolute(end_val_predicted[:,0] - end_val[:,0]))